In [45]:
from transformers import pipeline, AutoTokenizer
import torch
import json

In [46]:
# loading preliminary set of tags
final_tags = {
    # --- Primary Event & Opportunity Types ---
    "Workshop",
    "Lecture",
    "Seminar",
    "Talk",
    "Conference",
    "Forum",
    "Hackathon",
    "Olympiad",
    "Contest",
    "Festival",
    "Job Fair",
    "Master Class",
    "Club Meeting",
    "Ball",
    "Concert",
    "Party",
    "Quiz",
    "Game",
    "Internship",
    "Volunteering",
    # --- Common Topics ---
    "Programming",
    "Artificial Intelligence",
    "Computer Science",
    "Machine Learning",
    "Data Science",
    "Cybersecurity",
    "Robotics",
    "Science",
    "Mathematics",
    "Physics",
    "Business",
    "Startups",
    "Design",
    "Art",
    "Music",
    "Dance",
    "Sports",
    "Language Learning"
}

final_tags = list(final_tags)

# loading preprocessed messages
with open("data/o4u_preprocessed_messages_Jun_07_2025.json", "r", encoding="utf-8") as f:
    json_string = f.read()
event_texts = json.loads(json_string)

In [48]:
tokenizer = AutoTokenizer.from_pretrained('sileod/deberta-v3-base-tasksource-nli', model_max_length=512)

classifier = pipeline(
    "zero-shot-classification",
    model='sileod/deberta-v3-base-tasksource-nli',
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

def get_relevant_tags(event_description, all_tags, threshold):
    '''
    Identifies relevant tags for an event description using zero-shot classification.
    Returns tags with confidence scores above the specified threshold.

    Args:
        event_description (str): Text description of the event to classify
        all_tags (list):         Candidate tags for classification
        threshold (float):       Minimum confidence score (0.0-1.0) for tag inclusion

    Returns:
        list: Tuples of (confidence_score, tag) filtered by threshold,
               sorted by descending confidence (order from classifier output)
    '''
    result = classifier(
        event_description,
        candidate_labels=all_tags,
        truncation=True,
        multi_label=True
    )
    
    relevant_tags = [
        (score, tag) for tag, score in zip(result['labels'], result['scores'])
        if score >= threshold
    ]
    
    return relevant_tags

    
for desc in event_texts[:5]:
        selected_tags = get_relevant_tags(
            event_description=desc,
            all_tags=final_tags,
            threshold=0.8 # empirically chosen
        )
        print(desc, "\nTags:", selected_tags, '\n')

Device set to use cpu


Dear students, This channel advertises minor extracurricular activities, internal and external events, hackathons, competitions, campaigns and other potentially interesting happenings. All mentioned is supposed to help you to keep informed about additional opportunities for own personal and professional development. Keep in touch! 
Tags: [(0.8832920789718628, 'Hackathon'), (0.8111681342124939, 'Forum')] 

Hi there! Student Affairs is urgently looking for 3 volunteers to help with administrative work today from 15:30 until 18:00. Your efforts will be compensated with: - innopoints - tea cookies, if you like - friendly 319 team - amazing reputation in the future! If you may help please message andrejsblakunovs 
Tags: [(0.9704569578170776, 'Volunteering')] 

Hi there! Want any of these? Student Affairs are looking for volunteers to help with administrative work - today 15:00-17:00 or - tomorrow in 319 from 14:00 to 16:00. Your efforts will be compensated with: - IBC 2019 T-shirt - tea coo